## Project : Automate SAP Extraction File by Python
### Concept : 
#### ✅Step 1 : Establish the connection & Log in SAP
#### ✅Step 2 : Get access to the transaction and extract file (with a dynamic period)
#### ✅Step 3 : Close SAP application & Excel Workbook
#### ✅Step 4 : Data Cleaning
#### ✅Step 5 : File Transform and File Extraction



## Step 1 : Establish the connection & Log in SAP

In [30]:
import subprocess
import time
import win32com.client
from datetime import datetime, timedelta  # Import timedelta

# Define the SAP GUI path
sap_path = r'C:\Program Files (x86)\SAP\FrontEnd\SapGui\saplogon.exe'

# Function to start the SAP GUI application
def start_sap_gui(sap_path):
    try:
        subprocess.Popen(sap_path)
        print("SAP GUI started successfully.")
        # Wait for a few seconds to ensure SAP GUI is fully loaded
        time.sleep(7)
    except Exception as e:
        print(f"Error starting SAP GUI: {e}")

# Function to connect to SAP
def connect_to_sap():
    try:
        sap_gui_auto = win32com.client.GetObject("SAPGUI")
        if not sap_gui_auto:
            print("SAP GUI not found")
            return None
        
        application = sap_gui_auto.GetScriptingEngine
        connection = application.OpenConnection("your connection name", True)
        session = connection.Children(0)
        return session
    except Exception as e:
        print(f"Error connecting to SAP: {e}")
        return None

# Function to log in to SAP
def login_to_sap(session, client, username, password, language):
    try:
        session.findById("wnd[0]").maximize()
        session.findById("wnd[0]/usr/txtRSYST-MANDT").text = client
        session.findById("wnd[0]/usr/txtRSYST-BNAME").text = username
        session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = password
        session.findById("wnd[0]/usr/txtRSYST-LANGU").text = language
        session.findById("wnd[0]/usr/txtRSYST-LANGU").setFocus()
        session.findById("wnd[0]/usr/txtRSYST-LANGU").caretPosition = 2
        session.findById("wnd[0]").sendVKey(0)

        # Check if the "continue logon" popup appears and select the appropriate option
        try:
            continue_logon_dialog = session.findById("wnd[1]/usr/radMULTI_LOGON_OPT2")
            if continue_logon_dialog:
                continue_logon_dialog.select()
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
        except:
            pass

        print("Logged in to SAP successfully.")
    except Exception as e:
        print(f"Error logging in to SAP: {e}")

# Start SAP GUI
start_sap_gui(sap_path)

# Connect to SAP
sap_session = connect_to_sap()

# If connection is successful, log in to SAP and perform actions
if sap_session:
    login_to_sap(sap_session, "your client", "your username", "your password", "EN")


SAP GUI started successfully.
Logged in to SAP successfully.


## Step 2 : Get access to the transaction and extract file (with a dynamic period)

In [31]:
from datetime import datetime, timedelta
import time

# Calculate the required dates
today = datetime.today()
first_day_this_month = today.replace(day=1)
last_day_last_month = first_day_this_month - timedelta(days=1)
first_day_last_month = last_day_last_month.replace(day=1)
start_date = first_day_last_month - timedelta(days=1)  # last day of the month before last month
start_date = start_date.replace(day=26)  # set to 26th of the month before last month
end_date = last_day_last_month.replace(day=25)  # set to 25th of last month

# Convert dates to the required format
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

def wait_for_element(session, element_id, timeout=30):
    """Wait for an element to be available in the session."""
    for _ in range(timeout):
        try:
            element = session.findById(element_id)
            return element
        except:
            time.sleep(1)
    raise Exception(f"Element with ID {element_id} not found within {timeout} seconds")

try:
    # Initialize SAP GUI scripting objects
    SapGuiAuto = win32com.client.GetObject("SAPGUI")
    application = SapGuiAuto.GetScriptingEngine
    connection = application.Children(0)
    session = connection.Children(0)

    # Perform the needed steps in SAP
    session.findById("wnd[0]").maximize()
    wait_for_element(session, "wnd[0]/usr/cntlIMAGE_CONTAINER/shellcont/shell/shellcont[0]/shell").selectedNode = "0000000028"
    session.findById("wnd[0]/usr/cntlIMAGE_CONTAINER/shellcont/shell/shellcont[0]/shell").doubleClickNode("0000000028")
    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    session.findById("wnd[0]/tbar[1]/btn[6]").press()
    session.findById("wnd[1]/usr/cntlCONTAINER_4300/shellcont/shell").firstVisibleRow = 128
    session.findById("wnd[1]/usr/cntlCONTAINER_4300/shellcont/shell").firstVisibleRow = 203
    session.findById("wnd[1]/usr/cntlCONTAINER_4300/shellcont/shell").currentCellRow = 218
    session.findById("wnd[1]/usr/cntlCONTAINER_4300/shellcont/shell").selectedRows = "218"
    session.findById("wnd[1]/usr/cntlCONTAINER_4300/shellcont/shell").doubleClickCurrentCell()
    session.findById("wnd[1]/tbar[0]/btn[0]").press()

    # Set the evaluation period
    session.findById("wnd[0]/usr/subSUB_MAIN:SAPLAQ_ADHOC:0210/subSUB_APPLICATION:SAPLHR_QUERY_APPL_AREA:0122/subSUB:SAPLHR_QUERY_APPL_AREA:0121/btnDYNP121-EVALUATION_PERIOD_TEXT").press()
    session.findById("wnd[0]/usr/subSUB_MAIN:SAPLAQ_ADHOC:0210/subSUB_APPLICATION:SAPLHR_QUERY_APPL_AREA:0122/subSUB:SAPLHR_QUERY_APPL_AREA:0120/ctxtG_APPL_AREA-BEGDA").setFocus()
    session.findById("wnd[0]/usr/subSUB_MAIN:SAPLAQ_ADHOC:0210/subSUB_APPLICATION:SAPLHR_QUERY_APPL_AREA:0122/subSUB:SAPLHR_QUERY_APPL_AREA:0120/ctxtG_APPL_AREA-BEGDA").caretPosition = 5
    session.findById("wnd[0]").sendVKey(4)
    session.findById("wnd[1]/usr/cntlCONTAINER/shellcont/shell").selectionInterval = f"{start_date_str},{start_date_str}"

    session.findById("wnd[0]/usr/subSUB_MAIN:SAPLAQ_ADHOC:0210/subSUB_APPLICATION:SAPLHR_QUERY_APPL_AREA:0122/subSUB:SAPLHR_QUERY_APPL_AREA:0120/ctxtG_APPL_AREA-ENDDA").setFocus()
    session.findById("wnd[0]/usr/subSUB_MAIN:SAPLAQ_ADHOC:0210/subSUB_APPLICATION:SAPLHR_QUERY_APPL_AREA:0122/subSUB:SAPLHR_QUERY_APPL_AREA:0120/ctxtG_APPL_AREA-ENDDA").caretPosition = 7
    session.findById("wnd[0]").sendVKey(4)
    session.findById("wnd[1]/usr/cntlCONTAINER/shellcont/shell").selectionInterval = f"{end_date_str},{end_date_str}"

    # Execute and save the report
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    session.findById("wnd[0]/mbar/menu[0]/menu[4]/menu[1]").select()
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = r"your destination file path"
    session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
    session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 50
    session.findById("wnd[1]/tbar[0]/btn[11]").press()
    session.findById("wnd[0]").sendVKey(3)
    session.findById("wnd[0]").sendVKey(3)

except Exception as e:
    print(f"Error during SAP GUI operations: {e}")
    raise


## Step 3 : Close SAP application & Excel Workbook

In [32]:
# Close the SAP app

import win32com.client
import time

def close_sap_sessions():
    try:
        # Initialize SAP GUI scripting objects
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        application = SapGuiAuto.GetScriptingEngine
        connection = application.Children(0)

        # Iterate through all open sessions and close each one
        for i in range(connection.Children.Count - 1, -1, -1):
            try:
                session = connection.Children(i)
                session.findById("wnd[0]").close()

                # Check if a logoff confirmation dialog appears and handle it
                try:
                    session.findById("wnd[1]/usr/btnSPOP-OPTION1").press()
                except:
                    pass

            except Exception as e:
                print(f"An error occurred while closing session {i}: {e}")

        print("All SAP sessions closed.")
    except Exception as e:
        print(f"An error occurred: {e}")

def close_sap_logon():
    try:
        # Connect to the SAP Logon window using pywinauto
        from pywinauto.application import Application
        app = Application(backend="uia").connect(title_re="SAP Logon*")
        sap_logon_window = app.top_window()
        sap_logon_window.close()

        print("SAP Logon application closed.")
    except Exception as e:
        print(f"An error occurred while closing SAP Logon: {e}")

if __name__ == "__main__":
    close_sap_sessions()
    time.sleep(3)
    close_sap_logon()
    
## Wait for Excel File Open 

time.sleep(5)

# Close Excel App
import win32com.client

try:
    # Create a connection to the Excel application
    excel_app = win32com.client.Dispatch("Excel.Application")

    # Check if Excel is running by accessing the Workbooks property
    if excel_app.Workbooks.Count > 0:
        # Close all open workbooks without saving changes
        for workbook in excel_app.Workbooks:
            workbook.Close(False)  # False corresponds to not saving changes
    
    # Quit the Excel application
    excel_app.Quit()
    print("Excel application closed.")
except Exception as e:
    print(f"An error occurred: {e}")


All SAP sessions closed.
SAP Logon application closed.
Excel application closed.


## Step 4: Data Cleaning

In [33]:
import pandas as pd

file_path = r'your_file_path_raw'
df = pd.read_excel(file_path)

df['Birth date'] = pd.to_datetime(df['Birth date'])
            
            # Format the 'Birth date' column to dd-mm (hide the year)
df['Birth date'] = df['Birth date'].dt.strftime('%d-%m')

        # Sort the DataFrame by 'Pers.No.' and 'End Date' to ensure we get the latest end date for each 'Pers.No.'
df_sorted = df.sort_values(by=['Pers.No.', 'End Date'], ascending=[True, False])

        # Mark the rows to keep and remove based on the latest 'End Date' for each 'Pers.No.'
df_sorted['Action'] = df_sorted.duplicated(subset=['Pers.No.'], keep='first').apply(lambda x: 'Remove' if x else 'Keep')

        # Remove rows where 'Action' is 'Remove' or 'TU Premium' is 0
df_check = df_sorted[(df_sorted['Action'] != 'Remove') & (df_sorted['TU Premium'] != 0)]

columns_to_remove = ['Crcy', 'Start Date', 'End Date', 'Action']

df_final = df_check.drop(columns=columns_to_remove)

## Step 5: File Transform and File Extraction

In [ ]:
import pandas as pd
from datetime import datetime
try:
    import xlsxwriter
except ModuleNotFoundError:
    import os
    os.system('pip install xlsxwriter')
    import xlsxwriter

current_date = datetime.now()
formatted_date = current_date.strftime('%Y%m')
output_file_name = f'{formatted_date} your file_final_name'
output_file_path = f'your_desitnation_file_path{output_file_name}'

with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    df_final.to_excel(writer, index=False, sheet_name='Sheet1')
    
    # Access the XlsxWriter workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Adjust the width of each column to fit the contents
    for column in df_final:
        column_width = max(df_final[column].astype(str).map(len).max(), len(column))
        col_idx = df_final.columns.get_loc(column)
        worksheet.set_column(col_idx, col_idx, column_width)
center_format = workbook.add_format({'align': 'center'})
birth_date_col_idx = df_final.columns.get_loc('Birth date')
worksheet.set_column(birth_date_col_idx, birth_date_col_idx, column_width, center_format)

for row_num, value in enumerate(df_final['Birth date'], start=1):
        worksheet.write(row_num, birth_date_col_idx, value, center_format)

print(f"File saved to {output_file_path}")